In [3]:
import requests
import pandas as pd
import geopandas as gpd
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
from shapely import wkt
import folium
from folium import LayerControl, GeoJson, GeoJsonPopup, GeoJsonTooltip
import branca
import plotly.express as px  # Import Plotly Express for plotting

# Fetch data from the Flask API
def fetch_data():
    try:
        response = requests.get('http://127.0.0.1:5000/map')
        response.raise_for_status()
        data = response.json()
        return pd.DataFrame(data)
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return pd.DataFrame()

# Function to determine popup content based on risk type and level
def get_popup_content(risk_type, risk_level, selected_column):
    if risk_type == 'landslide_surface_area':
        label = 'Landslide Surface Area'
        unit = ' (km²)'
    elif risk_type == 'population':
        label = 'Population'
        unit = ' (Number)'
    elif risk_type == 'buildings':
        label = 'Buildings'
        unit = ' (Number)'
    else:
        label = 'Data'
        unit = ''

    if risk_level == 'very_high':
        level = 'very high'
    elif risk_level == 'high':
        level = 'high'
    elif risk_level == 'medium':
        level = 'medium'
    else:
        level = 'Risk Level'

    return f"{label} at {level} risk{unit}"

# Initialize the Dash app
app = Dash(__name__)

# Define base tile layers with attribution and graceful names
base_tiles = {
    'OpenStreetMap': {
        'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png',
        'attr': 'Map data &copy; OpenStreetMap contributors',
        'name': 'OpenStreetMap'
    },
    'Esri Topo': {
        'url': 'https://{s}.tile.opentopomap.org/{z}/{x}/{y}.png',
        'attr': 'Map data &copy; OpenTopoMap contributors',
        'name': 'Esri Topo'
    },
    'CartoDB Voyager': {
        'url': 'https://{s}.basemaps.cartocdn.com/rastertiles/voyager/{z}/{x}/{y}{r}.png',
        'attr': 'Map tiles by Carto, under CC BY 3.0. Data by OpenStreetMap, under ODbL.',
        'name': 'CartoDB Voyager'
    },
    'CartoDB Dark': {
        'url': 'https://{s}.basemaps.cartocdn.com/dark_all/{z}/{x}/{y}{r}.png',
        'attr': 'Map tiles by Carto, under CC BY 3.0. Data by OpenStreetMap, under ODbL.',
        'name': 'CartoDB Dark'
    }
}

# Define the layout
app.layout = html.Div(
    style={
        'backgroundColor': '#f0f0f5',  # Setting a light grey background
        'padding': '20px'
    },
    children=[
        html.H1('Landslide Risk Dashboard', style={'text-align': 'center'}),

        html.Div([
            html.Div([
                html.Label('Risk Type:', style={'font-weight': 'bold', 'margin-right': '10px'}),
                dcc.Dropdown(
                    id='risk-type-dropdown',
                    options=[
                        {'label': 'Landslide Surface Area', 'value': 'landslide_surface_area'},
                        {'label': 'Population', 'value': 'population'},
                        {'label': 'Buildings', 'value': 'buildings'}
                    ],
                    value='landslide_surface_area',
                    clearable=False,  # Ensure dropdown is not clearable
                    style={'width': '100%', 'font-size': '14px', 'padding': '8px', 'margin-right': '10px',
                           'border': '1px solid #ccc', 'border-radius': '4px', 'color': '#333'}
                ),
            ], style={'display': 'inline-block', 'width': '30%', 'vertical-align': 'top', 'padding': '10px'}),

            html.Div([
                html.Label('Risk Level:', style={'font-weight': 'bold', 'margin-right': '10px'}),
                dcc.Dropdown(
                    id='risk-level-dropdown',
                    options=[
                        {'label': 'Very High', 'value': 'very_high'},
                        {'label': 'High', 'value': 'high'},
                        {'label': 'Medium', 'value': 'medium'}
                    ],
                    value='very_high',
                    clearable=False,  # Ensure dropdown is not clearable
                    style={'width': '100%', 'font-size': '14px', 'padding': '8px', 'margin-right': '10px',
                           'border': '1px solid #ccc', 'border-radius': '4px', 'color': '#333'}
                ),
            ], style={'display': 'inline-block', 'width': '30%', 'vertical-align': 'top', 'padding': '10px'}),

            html.Div([
                html.Label('Base Tile Layer:', style={'font-weight': 'bold', 'margin-right': '10px'}),
                dcc.Dropdown(
                    id='base-tile-dropdown',
                    options=[{'label': tile_name, 'value': tile_info['url']} for tile_name, tile_info in base_tiles.items()],
                    value=base_tiles['OpenStreetMap']['url'],
                    style={'width': '100%', 'font-size': '14px', 'padding': '8px', 'margin-right': '10px',
                           'border': '1px solid #ccc', 'border-radius': '4px', 'color': '#333'}
                ),
            ], style={'display': 'inline-block', 'width': '30%', 'vertical-align': 'top', 'padding': '10px'}),
        ]),

        html.Div([
            html.Label('Opacity:', style={'font-weight': 'bold', 'margin-right': '10px'}),
            dcc.Slider(
                id='opacity-slider',
                min=0,
                max=1,
                step=0.01,
                value=0.7,
                marks={0: '0', 1: '1'},
                tooltip={'always_visible': True, 'placement': 'bottom'}
            ),
            html.Div(id='opacity-output-container', style={'margin-top': '10px'})
        ], style={'width': '30%', 'display': 'inline-block', 'vertical-align': 'top', 'padding': '20px', 'text-align': 'center'}),

        html.Div([
            html.Div(id='map-container', children=[
                html.Iframe(id='map', srcDoc=None, width='100%', height='600')
            ])
        ]),

        html.Div([
            dcc.Graph(id='bar-chart')
        ])
    ]
)

# Callback to update the map and opacity
@app.callback(
    Output('map', 'srcDoc'),
    [Input('risk-type-dropdown', 'value'),
     Input('risk-level-dropdown', 'value'),
     Input('opacity-slider', 'value'),
     Input('base-tile-dropdown', 'value')]
)
def update_map(risk_type, risk_level, opacity, base_tile_url):
    df = fetch_data()

    if df.empty:
        return None  # Return None if data fetching failed

    # Convert WKT to GeoDataFrame
    df['geometry'] = df['geom_wkt'].apply(wkt.loads)
    gdf = gpd.GeoDataFrame(df, geometry='geometry')

    # Set the initial CRS to EPSG:32632 and then transform to EPSG:4326
    gdf.set_crs('EPSG:32632', inplace=True)
    gdf = gdf.to_crs(epsg=4326)

    # Drop the 'geom_wkt' column
    gdf = gdf.drop(columns=['geom_wkt'])

    # Map risk type and level to the corresponding column
    risk_columns = {
        'landslide_surface_area': {
            'very_high': 'ar_fr_p4',
            'high': 'ar_fr_p3',
            'medium': 'ar_fr_p2'
        },
        'population': {
            'very_high': 'pop_fr_p4',
            'high': 'pop_fr_p3',
            'medium': 'pop_fr_p2'
        },
        'buildings': {
            'very_high': 'ed_fr_p4',
            'high': 'ed_fr_p3',
            'medium': 'ed_fr_p2'
        }
    }

    selected_column = risk_columns[risk_type][risk_level]

    # Define custom colormaps for each risk type using ColorBrewer palettes
    colormaps = {
        'landslide_surface_area': branca.colormap.LinearColormap(
            colors=['#f7fbff', '#c6dbef', '#6baed6', '#3182bd', '#08519c'],
            vmin=gdf[selected_column].min(),
            vmax=gdf[selected_column].max(),
            caption="Landslide Surface Area Risk Level (km²) "
        ),
        'population': branca.colormap.LinearColormap(
            colors=['#f7fcf5', '#c7e9c0', '#74c476', '#31a354', '#006d2c'],
            vmin=gdf[selected_column].min(),
            vmax=gdf[selected_column].max(),
            caption="Population Risk Level (Number)"
        ),
        'buildings': branca.colormap.LinearColormap(
            colors=['#fff5f0', '#fcbba1', '#fc9272', '#fb6a4a', '#cb181d'],
            vmin=gdf[selected_column].min(),
            vmax=gdf[selected_column].max(),
            caption="Buildings Risk Level (Number)"
        )
    }

    colormap = colormaps[risk_type]

    # Get the attribution for the selected base tile
    base_tile_info = next(tile_info for tile_info in base_tiles.values() if tile_info['url'] == base_tile_url)
    base_tile_attr = base_tile_info['attr']
    base_tile_name = base_tile_info['name']

    # Create a map
    m = folium.Map(location=[45.05780847800959, 9.635009765625002], zoom_start=8, name=base_tile_name, tiles=base_tile_url, attr=base_tile_attr)

    # Add a choropleth layer to the map
    def style_function(feature):
        return {
            'fillColor': colormap(feature['properties'][selected_column]),
            'color': 'black',
            'fillOpacity': opacity,
            'weight': 0.5
        }

    # Create GeoJsonPopup with dynamic content
    popup = GeoJsonPopup(
        fields=['nome', selected_column],
        aliases=['Region', get_popup_content(risk_type, risk_level, selected_column)],  # Use function to get dynamic content
        localize=True,
        labels=True,
        style="background-color: yellow;",
    )

    # Create GeoJsonTooltip with dynamic content
    tooltip = GeoJsonTooltip(
        fields=['nome', selected_column, "ar_kmq"],
        aliases=['Region', get_popup_content(risk_type, risk_level, selected_column), "Total Area (km²):"],
        localize=True,
        sticky=True,
        style="""
            background-color: #F0EFEF;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
        max_width=800,
    )

    # Add GeoJson layer with popup and tooltip to the map
    g = GeoJson(
        gdf,
        name='Landslide Risk Choropleth',
        style_function=style_function,
        popup=popup,
        highlight_function=lambda x: {'weight':2, 'fillOpacity': 1.0},
        tooltip=tooltip,
    ).add_to(m)

    # Add layer control and colormap to the map
    LayerControl(position='topleft').add_to(m)
    colormap.add_to(m)

    # Return the HTML representation of the map
    return m._repr_html_()

# Callback to update the bar chart
@app.callback(
    Output('bar-chart', 'figure'),
    [Input('risk-type-dropdown', 'value'),
     Input('risk-level-dropdown', 'value')]
)
def update_bar_chart(risk_type, risk_level):
    df = fetch_data()

    if df.empty:
        return {}

    # Map risk type and level to the corresponding column
    risk_columns = {
        'landslide_surface_area': {
            'very_high': 'ar_fr_p4',
            'high': 'ar_fr_p3',
            'medium': 'ar_fr_p2'
        },
        'population': {
            'very_high': 'pop_fr_p4',
            'high': 'pop_fr_p3',
            'medium': 'pop_fr_p2'
        },
        'buildings': {
            'very_high': 'ed_fr_p4',
            'high': 'ed_fr_p3',
            'medium': 'ed_fr_p2'
        }
    }

    selected_column = risk_columns[risk_type][risk_level]

    # Define custom colors for the bar chart based on risk type
    bar_colors = {
        'landslide_surface_area': '#3182bd',  # Blue
        'population': '#31a354',  # Green
        'buildings': '#cb181d'  # Red
    }

    bar_color = bar_colors[risk_type]

    # Create a bar chart
    fig = px.bar(df, x='nome', y=selected_column,
                 labels={'nome': 'Region', selected_column: get_popup_content(risk_type, risk_level, selected_column)},
                 title=f'{risk_type.replace("_", " ").capitalize()} at {risk_level.replace("_", " ").capitalize()} Risk',
                 color_discrete_sequence=[bar_color])

    # Update y-axis label to format nicely
    unit_label = ' (km²)' if 'area' in selected_column else ' (Number)'
    y_axis_label = f"{risk_type.replace('_', ' ').capitalize()} at {risk_level.replace('_', ' ').capitalize()} Risk{unit_label}"
    fig.update_yaxes(title_text=y_axis_label)

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8081)
